In [2]:
# %load eval.py
from argparse import ArgumentParser

from pytorch_lightning import seed_everything, Trainer

from dataset.mpii_face_gaze_dataset import get_dataloaders
from train import Model

import numpy as np
import torch
from utils import calc_angle_error, PitchYawHelper
from sklearn.linear_model import LinearRegression

In [4]:
def simulate_calibration_lr(model, dataset, S=1, device='cuda'):
    """
    使用 Linear Regression 做視線偏移校正
    """
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        all_data = []
        for i in range(len(dataset)):
            item = dataset[i]
            data = {
                'left_eye': item['left_eye_image'].unsqueeze(0).to(device),
                'right_eye': item['right_eye_image'].unsqueeze(0).to(device),
                'face': item['full_face_image'].unsqueeze(0).to(device),
                'label': torch.tensor(
                    [float(item['gaze_pitch']), float(item['gaze_yaw'])],
                    dtype=torch.float32
                ).to(device),
                'person_idx': torch.tensor([[item['person_idx']]]).to(device)
            }
            all_data.append(data)

        # 選 S 張圖作為校正資料
        indices = np.random.choice(len(all_data), size=S, replace=False)
        calibration_set = [all_data[i] for i in indices]
        test_set = [all_data[i] for i in range(len(all_data)) if i not in indices]

        # 擷取 calibration 資料的 pred & label
        preds, labels = [], []
        for d in calibration_set:
            pred = model(
                d['person_idx'], d['face'], d['right_eye'], d['left_eye']
            ).squeeze(0).cpu().numpy()
            label = d['label'].cpu().numpy()
            preds.append(pred)
            labels.append(label)

        # 執行 Linear Regression 擬合 (pred -> label)
        reg = LinearRegression().fit(preds, labels)

        # 測試時套用 regression 校正
        angle_errors = []
        for d in test_set:
            pred = model(
                d['person_idx'], d['face'], d['right_eye'], d['left_eye']
            ).squeeze(0).cpu().numpy()
            corrected = reg.predict([pred])[0]
            angle = calc_angle_error(
                torch.tensor(corrected).unsqueeze(0).to(device),
                d['label'].unsqueeze(0)
            )
            angle_errors.append(angle.item())

        mean_err = np.mean(angle_errors)
        print(f"📐 Linear Calibration (S={S}): {mean_err:.2f} degrees")
        return mean_err


In [5]:
def simulate_calibration(model, dataset, S=1, device='cuda'):
    """
    模擬 S 張圖當校正資料，對剩下資料測試。
    模型 forward 接收格式：FinalModel(person_idx, full_face, right_eye, left_eye)
    """
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        all_data = []
        for i in range(len(dataset)):
            item = dataset[i]
            data = {
                'left_eye': item['left_eye_image'].unsqueeze(0).to(device),
                'right_eye': item['right_eye_image'].unsqueeze(0).to(device),
                'face': item['full_face_image'].unsqueeze(0).to(device),
                'label': torch.tensor(
                    [float(item['gaze_pitch']), float(item['gaze_yaw'])],
                    dtype=torch.float32
                ).to(device),
                'person_idx': torch.tensor([[item['person_idx']]]).to(device)  # shape [1, 1]
            }
            all_data.append(data)

        # Step 1：選 S 張圖當校正資料
        indices = np.random.choice(len(all_data), size=S, replace=False)
        calibration_set = [all_data[i] for i in indices]
        test_set = [all_data[i] for i in range(len(all_data)) if i not in indices]

        # Step 2：計算 bias
        cal_outputs, cal_labels = [], []
        for d in calibration_set:
            pred = model(
                d['person_idx'],
                d['face'],
                d['right_eye'],
                d['left_eye']
            ).squeeze(0)
            cal_outputs.append(pred)
            cal_labels.append(d['label'])

        bias = torch.stack(cal_labels).mean(dim=0) - torch.stack(cal_outputs).mean(dim=0)

        # Step 3：套用 bias 做測試
        angle_errors = []
        for d in test_set:
            pred = model(
                d['person_idx'],
                d['face'],
                d['right_eye'],
                d['left_eye']
            ).squeeze(0) + bias

            angle = calc_angle_error(pred.unsqueeze(0), d['label'].unsqueeze(0))
            angle_errors.append(angle.item())

        mean_err = np.mean(angle_errors)
        print(f"📐 Calibration (S={S}): {mean_err:.2f} degrees")
        return mean_err

In [6]:
if __name__ == '__main__':

    ###非變動區塊
    k = [1,5,9,16,32,64, 128]
    adjust_slope = False
    ###對應作者表格grid calibration,如果是random就填true
    grid_calibration_samples=False
    batch_size =32
    path_to_data = './data/mpiifacegaze_preprocessed'
    
    
    ####變動區塊
    path_to_checkpoints = './saved_models/p00/p00_best-v28.ckpt'
    person_idx = 1
    validate_on_person = 0 
    
    seed_everything(42)

    print(f"{path_to_checkpoints}")
    
    model = Model.load_from_checkpoint(path_to_checkpoints, k=[9, 128], adjust_slope=False, grid_calibration_samples=False)

    trainer = Trainer(
        gpus=1,
        benchmark=True,
    )

    _, _, test_dataloader = get_dataloaders(        path_to_data, 
        validate_on_person, 
        person_idx, 
        batch_size   
    )
     
    trainer.test(model, test_dataloader)

    ####################拿出 test set（單人完整資料）
    test_dataset = test_dataloader.dataset
    #######################

    simulate_calibration_ls(model, test_dataset, S=1)
    simulate_calibration_ls(model, test_dataset, S=5)
    simulate_calibration_ls(model, test_dataset, S=9)
    simulate_calibration_ls(model, test_dataset, S=16)


Global seed set to 42


./saved_models/p00/p00_best-v28.ckpt


C:\Users\User\anaconda3\envs\gazegpu\lib\site-packages\pytorch_lightning\loops\utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


train on persons [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
valid on person 0
test on person 1
len(dataset_train) 60784
len(dataset_valid) 2927


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


len(dataset_test) 2904
Testing DataLoader 0: 100%|██████████| 181/181 [00:25<00:00,  7.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test/offset(k=0)/angular_error             3.6843724250793457
        test/offset(k=0)/loss                0.0026730604004114866
test/offset(k=128)/mean_angular_error         2.7850077686071395
test/offset(k=128)/std_angular_error          0.02994585057951566
 test/offset(k=9)/mean_angular_error          2.9401812864780426
 test/offset(k=9)/std_angular_error           0.1896842254231117
  test/offset(k=all)/angular_error            2.7732884883880615
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
📐 Least Squares (3D Ve